In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from util.bat import *
import random
import os
import pandas as pd
from sklearn.cluster import KMeans

# Set some Pandas options
pd.set_option('notebook_repr_html', True)
pd.set_option('max_columns', 30)
pd.set_option('max_rows', 20)

In [ ]:
def yang_cluster():
    info=extract_anabat('../data/P7132033.37#')
    t=info[0]
    freq=info[1]
    ampl=info[2]
    metadata=info[3]
    pulses=remove_noise2(t,freq)
    
    b,c=get_dy_dy2(pulses)
    
    bf=get_features(b)
    cf=get_features(c)
    
    bf.iloc[8:11,]
    cf.iloc[8:11,]
    ff=pd.concat([bf, cf.iloc[:,0:2]], axis=1)
    ff.iloc[8:11,]
    
    # Clustering a sample recording pulses using KMeans
    est = KMeans(3)  # 4 clusters
    # X=b.iloc[:,0:2]
    X=ff
    est.fit(X)
    y_kmeans = est.predict(X)
    plt.figure(figsize=(10,8))
    plt.scatter(X.iloc[:, 0], X.iloc[:, 1], c = y_kmeans, s=50, cmap='rainbow');

    display_pulses(pulses, len(pulses), 5, rand_flag = False, cluster = y_kmeans)
    return pulses, y_kmeans

In [ ]:
pulses, labels = yang_cluster()

pulses = pulses[:len(labels)]

for i, pulse in enumerate(pulses):
    if labels[i] == 2:
        labels[i] == 0

In [ ]:
# Following a tutorial from torres.ai

import keras
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.utils.vis_utils import model_to_dot
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from IPython.display import SVG

# Load data from MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Convert to float32 values between 0 and 1
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Reshape to vectors in R1 of length 784
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

# Reshape to vectors in R1 of length 10
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

model = Sequential()

model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dense(10, activation='softmax'))

model.compile(loss="categorical_crossentropy",
              optimizer="sgd",
              metrics = ['accuracy'])

model.fit(x_train, y_train, batch_size=10, epochs=40)
test_loss, test_acc = model.evaluate(x_test, y_test)
predictions = model.predict(x_test)
print(test_acc)

In [ ]:
# CNN tutorial
# https://towardsdatascience.com/convolutional-neural-networks-for-beginners-practical-guide-with-python-and-keras-dc688ea90dca

from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(train_images, train_labels,
          batch_size=32,
          epochs=5,
          verbose=1)

test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Test accuracy:', test_acc)

In [ ]:
import numpy as np
import glob
from PIL import Image
from random import shuffle
from keras import layers
from keras import models
from keras.utils import to_categorical

filenames = glob.glob('..\\..\\data\\sorted\\**\\*.png', recursive=True)
data = list()

label = {
        'echolocation': 0,
        'abnormal': 1,
        'noise': 2
    }

for filename in filenames:
    lbl = filename.rsplit('\\')[-2]    
    img = np.array(Image.open(filename))
    data.append([label.get(lbl), img])
    
shuffle(data)

train = data[:int(len(data)*.8)]
test = data[int(len(data)*.2):]

train_labels = [dat[0] for dat in train]
train_images = [dat[1] for dat in train]

test_labels = [dat[0] for dat in test]
test_images = [dat[1] for dat in test]

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model = models.Sequential()

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(train_images, train_labels, batch_size=100, epochs=5, verbose=1)

test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Test accuracy:', test_acc)


In [3]:
#https://adventuresinmachinelearning.com/python-tensorflow-tutorial/

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# Python optimisation variables
learning_rate = 0.5
epochs = 10
batch_size = 100

# declare the training data placeholders
# input x - for 28 x 28 pixels = 784
x = tf.placeholder(tf.float32, [None, 784])
# now declare the output data placeholder - 10 digits
y = tf.placeholder(tf.float32, [None, 10])

# now declare the weights connecting the input to the hidden layer
W1 = tf.Variable(tf.random_normal([784, 200], stddev=0.03), name='W1')
b1 = tf.Variable(tf.random_normal([200]), name='b1')

# and the weights connecting the hidden layer to the output layer
W2 = tf.Variable(tf.random_normal([200, 10], stddev=0.03), name='W2')
b2 = tf.Variable(tf.random_normal([10]), name='b2')

# calculate the output of the hidden layer
hidden_out = tf.add(tf.matmul(x, W1), b1)
hidden_out = tf.nn.relu(hidden_out)

# now calculate the hidden layer output - in this case, let's use a softmax activated output layer
y_ = tf.nn.softmax(tf.add(tf.matmul(hidden_out, W2), b2))
y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999) # prevent NaN from log(0)

#calculate how messed up the NN is
cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped) + (1 - y) * tf.log(1 - y_clipped), axis=1))

# add an optimiser
optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

# finally setup the initialisation operator
init_op = tf.global_variables_initializer()

# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# start the session
with tf.Session() as sess:
   # initialise the variables
   sess.run(init_op)
   total_batch = int(len(mnist.train.labels) / batch_size)
   for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
            _, c = sess.run([optimiser, cross_entropy], feed_dict={x: batch_x, y: batch_y})
            avg_cost += c / total_batch
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost))
   print(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels}))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [19]:
import tensorflow as tf
import numpy as np
import glob

datadir = '../../data/sorted/'
    
train = list()
for file in glob.glob(datadir + 'train_abnormal/*.png'):
    train.append([file, 0])

for file in glob.glob(datadir + 'train_echolocation/*.png'):
    train.append([file, 1])

test = list()
for file in glob.glob(datadir + 'test_abnormal/*.png'):
    test.append([file, 0])

for file in glob.glob(datadir + 'test_echolocation/*.png'):
    test.append([file, 1])


learning_rate = 0.5
epochs = 10
batch_size = 100

# Image to 1d array, label to 2vector
def _parse_function(filename, label):
    image_string = tf.read_file(filename)
    image = tf.image.decode_png(image_string, channels=1)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize_images(image, (256, 256))
    
    label = tf.one_hot(label, 2)
    return image, label

training = tf.data.Dataset.from_tensor_slices(([row[0] for row in train], [row[1] for row in train]))
testing = tf.data.Dataset.from_tensor_slices(([row[0] for row in test], [row[1] for row in test]))

training = training.map(_parse_function)
testing = testing.map(_parse_function)

iter = tf.data.Iterator.from_structure(training.output_types, training.output_shapes)
next_element = iter.get_next()
train_init_op = iter.make_initializer(training)
test_init_op = iter.make_initializer(testing)

def nn_model(in_data):
    c1 = tf.layers.conv2d(in_data, data_format = 'channels_last', filters = 96, kernel_size = (7, 7), strides = 3)
    a1 = tf.nn.relu(c1)
    m1 = tf.layers.MaxPooling2D(a1, pool_size = (4, 4), strides = 2)
    z1 = tf.keras.layers.ZeroPadding2D(m1, 1)
    
    c2 = tf.layers.conv2d(z1, 128, (3, 3), strides = 3)
    a2 = tf.nn.relu(c2)
    m2 = tf.layers.MaxPooling2D(a2, pool_size = (3, 3), strides = 2)
    z2 = tf.keras.layers.ZeroPadding2D(m2, 1)
    
    c3 = tf.layers.conv2d(z2, 256, (3, 3), strides = 3)
    
    f1 = tf.layers.Flatten()(c3)
    d1 = tf.layers.dense(f1, 256, activation = 'relu')
    dr1 = tf.layers.dropout(d1, rate = 0.5)
    d2 = tf.layers.dense(dr1, 1, activation = 'sigmoid')
    #bn = tf.layers.batch_normalization(in_data)
    #fc1 = tf.layers.dense(bn, 1000)
    #fc2 = tf.layers.dense(fc1, 300)
    #fc3 = tf.layers.dense(fc2, 2)
    #fc4 = tf.reduce_mean(fc3, axis = 0)
    #fc5 = tf.reduce_mean(fc4, axis = 0)
    #fc6 = tf.layers.dropout(fc5)
    #fc7 = tf.nn.relu(fc6)
    return fc7

print(next_element[0])
print(next_element[1])
# create the neural network model
logits = nn_model(next_element[0])
print(logits)

# add the optimizer and loss
loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=next_element[1], logits=logits))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

# get accuracy
correct_prediction = tf.equal(tf.argmax(next_element[1], 0), tf.argmax(logits, 0))
accuracy = tf.reduce_mean(tf.cast(tf.math.abs(tf.math.subtract(tf.cast(logits, tf.float32), next_element[1])), tf.float32))

init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_op)
    sess.run(train_init_op)
    
    for x in range(epochs):
        l, _, acc = sess.run([loss, optimizer, accuracy])
        print("Epoch: {}, loss: {:.3f}, training accuracy: {:.2f}%".format(x, l, acc * 100))
    
    sess.run(test_init_op)
    avg_acc = 0
    for i in range(100):
        acc = sess.run([accuracy])
        avg_acc += acc[0]
    
    print("Average validation set accuracy over {} iterations is {:.2f}%".format(100, (avg_acc / 100) * 100))
    

ValueError: 'size' must be a 1-D Tensor of 2 elements: new_height, new_width